In [1]:
from kafka import KafkaProducer,KafkaClient
from kafka.errors import KafkaError
import time
import json
import websocket
import logging
import re
#LOCAL IMPORTS
import config as myCred
from confluent_kafka.admin import AdminClient
import hashlib

In [2]:
logging.basicConfig(filename="logFile.log", level=logging.INFO)

In [3]:
#unit test to check if zookeeper an kafka are running properly
conf = {'bootstrap.servers':'localhost:9092'}
admin_client = AdminClient(conf)
topics = admin_client.list_topics().topics
if not topics: 
    logging.error(RuntimeError())
else:
    logging.info(topics)

In [4]:
#functions for API connnection

startTime =time.time()

def onOpen(ws):
    print("connection Opened")
    auth_data={
        "action": "authenticate",
        "data": {"key_id": myCred.API_KEY, "secret_key": myCred.SECRET_KEY}
    }
    
    ws.send(json.dumps(auth_data))
    
    listenMsg = {
        "action": "listen", 
        "data": {"streams": ["T.TSLA","T.AAPL","T.MSFT"]}
    }

    ws.send(json.dumps(listenMsg))
    
future =[]     
def onMsg(ws,message):
    if re.match('^.*authorized.*$',message):
        return
    else:
        if(time.time()-startTime)<30:
            print(message)
            try:
                future.append(producer.send(topicName, message.encode('utf-8')))
                return True
            except Exception as ex:
                #producer.close()
                logging.error(str(ex))
        else:
            producer.close()
            ws.close()
    
def onClose(ws):
    print("connection Closed")

In [5]:
socket = "wss://data.alpaca.markets/stream"
ws = websocket.WebSocketApp(socket,on_open=onOpen, on_close=onClose, on_message=onMsg)
try:
    producer=KafkaProducer(bootstrap_servers=" localhost:9092")
    topicName="stockData"
except KafkaError:
    logging.error(KafkaError)

In [6]:
ws.run_forever()

connection Opened
{"stream":"listening","data":{"streams":["T.TSLA","T.AAPL","T.MSFT"]}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"224","x":17,"p":1387.31,"s":2,"t":1594388521793000000,"c":[14,37,41],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"225","x":17,"p":1387.24,"s":8,"t":1594388521793000000,"c":[14,37,41],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"243","x":15,"p":212.15,"s":100,"t":1594388522306000000,"c":null,"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"397","x":17,"p":212.11,"s":2,"t":1594388523214000000,"c":[37],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"129","x":2,"p":1388.01,"s":2,"t":1594388523721000000,"c":[37],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"108","x":3,"p":212.07,"s":26,"t":1594388525862000000,"c":[14,37,41],"z":3}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"368","x":2,"p":380.87,"s":100,"t":1594388526226000000,"c":null,"z":3}}
{"stream":"T.MSFT","data":{"ev":"T

In [7]:
try:
    ret = [f.get(timeout=30) for f in future]
    logging.info(ret[-1].offset)
except KafkaError:
    logging.info(KafkaError)